*** TO DO FOR CATAN: ***
RAINBOW: 
    1. vs Random
    2. vs Weighted Random
    3. vs MTCS
    4. vs Victory Point
    5. vs AlphaBeta
Masked PPO the same 
NFSP 
MuZero

MUZERO

In [ ]:
import copy
import sys

from modules.muzero_world_model import MuzeroWorldModel

sys.path.append("../../")

from modules.utils import CategoricalCrossentropyLoss, KLDivergenceLoss


from agents.catan_player_wrapper import CatanPlayerWrapper

from agents.muzero_agent_torch import MuZeroAgent
from agent_configs import MuZeroConfig
from muzero.action_functions import action_as_onehot, action_as_plane

from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer
from wrappers import record_video_wrapper

from game_configs.game_2048_config import Game2048Config
from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)

from torch.optim import Adam, SGD
import custom_gym_envs

env = Game2048Config().make_env()
params = {
    "num_simulations": 100,
    "per_alpha": 0.6,
    "per_beta": 0.4,
    "per_beta_final": 0.4,
    "n_step": 10,
    "residual_layers": [(64, 3, 1)],
    "dense_layer_widths": [],
    "reward_dense_layer_widths": [32],
    "reward_conv_layers": [(16, 1, 1)],
    "chance_dense_layer_widths": [32],
    "chance_conv_layers": [(16, 1, 1)],
    "actor_dense_layer_widths": [32],
    "actor_conv_layers": [(16, 1, 1)],
    "critic_dense_layer_widths": [32],
    "critic_conv_layers": [(16, 1, 1)],
    "to_play_dense_layer_widths": [],
    "to_play_conv_layers": [],
    # "known_bounds": [-1, 1],
    "support_range": 300,
    "minibatch_size": 512,  # 1024
    "replay_buffer_size": 1e6,
    # "lr_ratio": float("inf"),
    # "lr_ratio": 0.1,
    "learning_rate": 0.003,
    "weight_decay": 1e-4,
    "num_workers": 4,  # 1000
    "min_replay_buffer_size": 1024,
    "root_dirichlet_alpha": 0.3,
    "root_exploration_fraction": 0.25,
    "optimizer": Adam,
    # "temperature_updates": [1e4, 2e4, 3e4],  # [1e5, 2e5, 3e5],
    # "temperatures": [1.0, 0.5, 0.1, 0.0],
    "temperature_updates": [1e5],  # [1e5, 2e5, 3e5],
    "temperatures": [0.25, 0.0],
    "temperature_with_training_steps": True,
    "to_play_loss_factor": 0.0,
    "transfer_interval": 1,
    "gumbel": False,
    "gumbel_m": 8,
    "policy_loss_function": CategoricalCrossentropyLoss(),
    "value_loss_function": CategoricalCrossentropyLoss(),
    "reward_loss_function": CategoricalCrossentropyLoss(),
    "training_steps": 20e6,
    "reanalyze_ratio": 0.0,
    "reanalyze_noise": True,
    "value_loss_factor": 0.25,
    "injection_frac": 0.25,
    "reanalyze_method": "mcts",
    "consistency_loss_factor": 2.0,  # might want to be zero for "average state" planning, as initial is known, but one step forward in dynamics has some randomness
    "projector_output_dim": 1024,
    "projector_hidden_dim": 1024,
    "predictor_output_dim": 1024,
    "predictor_hidden_dim": 512,
    "value_prefix": True,
    "lstm_horizon_len": 5,
    "lstm_hidden_size": 64,
    "q_estimation_method": "v_mix",
    "clipnorm": 10,
    "stochastic": True,
    "vqvae_commitment_cost_factor": 1.0,
    "discount_factor": 0.999,
    "use_true_chance_codes": True,
    "world_model_cls": MuzeroWorldModel,
}
game_config = Game2048Config()
config = MuZeroConfig(config_dict=params, game_config=game_config)


agent = MuZeroAgent(
    env=env,
    config=config,
    name="2048_muzero_everything_2",
    device="cpu",
    test_agents=[],
)
agent.checkpoint_interval = 100
agent.test_interval = 1000
agent.test_trials = 25

agent.train()

In [ ]:
import gymnasium as gym

env = Game2048Config().make_env()
print(isinstance(env.observation_space, gym.spaces.Box))
# isinstance(obs_space, gym.spaces.Box)
print(env.reset())

In [ ]:
import copy
import sys

sys.path.append("../../")

from modules.utils import CategoricalCrossentropyLoss, KLDivergenceLoss


from agents.catan_player_wrapper import CatanPlayerWrapper

from agents.muzero_agent_torch import MuZeroAgent
from agent_configs import MuZeroConfig
from muzero.action_functions import action_as_onehot, action_as_plane

from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer
from wrappers import record_video_wrapper

from game_configs.catan_config import CatanConfig
from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)

from torch.optim import Adam, SGD

env = CatanConfig().make_env(
    num_players=2,
    map_type="BASE",
    vps_to_win=10,
    representation="vector",
    invalid_action_reward=-10,
    render_mode="rgb_array",
)
env = record_video_wrapper(copy.deepcopy(env), "./videos", 1)
params = {
    "num_simulations": 50,
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "n_step": 2000,
    "residual_layers": [],
    "dense_layer_widths": [64] * 3,
    "reward_dense_layer_widths": [16],
    "reward_conv_layers": [],
    "chance_dense_layer_widths": [16],
    "chance_conv_layers": [],
    "actor_dense_layer_widths": [16],
    "actor_conv_layers": [],
    "critic_dense_layer_widths": [16],
    "critic_conv_layers": [],
    "to_play_dense_layer_widths": [16],
    "to_play_conv_layers": [],
    "known_bounds": [-1, 1],
    "support_range": None,
    "minibatch_size": 256,
    "replay_buffer_size": 200000,
    # "lr_ratio": float("inf"),
    "lr_ratio": 0.1,
    "learning_rate": 3e-4,
    "num_workers": 4,
    "min_replay_buffer_size": 3000,
    "root_dirichlet_alpha": 0.03,
    "optimizer": Adam,  # SGD
    "temperature_updates": [4],
    "to_play_loss_factor": 2.0,
    "transfer_interval": 1,
    "gumbel": True,
    "gumbel_m": 8,
    "policy_loss_function": KLDivergenceLoss(),
    "training_steps": 200000,
    "reanalyze_ratio": 0.0,
    "reanalyze_noise": True,  # for gumbel
    "value_loss_factor": 1.0,  # for reanalyze
    "injection_frac": 0.25,
    "reanalyze_method": "mcts",
    "consistency_loss_factor": 0.0,  # might want to be zero for "average state" planning, as initial is known, but one step forward in dynamics has some randomness
    "projector_output_dim": 128,
    "projector_hidden_dim": 128,
    "predictor_output_dim": 128,
    "predictor_hidden_dim": 64,
    "value_prefix": True,
    "lstm_horizon_len": 5,
    "lstm_hidden_size": 64,
    "q_estimation_method": "v_mix",
    "clipnorm": 10,
    "stochastic": True,
    "vqvae_commitment_cost_factor": 0.01,
}
game_config = CatanConfig()
config = MuZeroConfig(config_dict=params, game_config=game_config)


agent = MuZeroAgent(
    env=env,
    config=config,
    name="catan_tournament_map_4",
    device="cpu",
    test_agents=[
        CatanPlayerWrapper(RandomPlayer, Color.BLUE),
        CatanPlayerWrapper(WeightedRandomPlayer, Color.BLUE),
        # CatanPlayerWrapper(AlphaBetaPlayer, Color.BLUE),
    ],
)
agent.checkpoint_interval = 1
agent.test_interval = 100
agent.test_trials = 15

agent.train()

In [ ]:
import torch

mse_loss = torch.nn.MSELoss()

t = torch.tensor(
    [[[1, 1, 1], [1, 1, 1], [1, 1, 1]], [[1, 1, 1], [1, 1, 1], [1, 1, 1]]]
).float()
t1 = (
    torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]], [[1, 1, 1], [1, 1, 1], [1, 1, 1]]])
    - 1
).float()

print(mse_loss(t, t1))

In [ ]:
import copy
import sys

from utils import KLDivergenceLoss


sys.path.append("../../")
from agents.catan_player_wrapper import CatanPlayerWrapper

from agents.muzero_agent_torch import MuZeroAgent
from agent_configs import MuZeroConfig
from muzero.action_functions import action_as_onehot, action_as_plane

from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer
from utils.utils import record_video_wrapper

from game_configs.catan_config import CatanConfig
from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)

from torch.optim import Adam, SGD

env = CatanConfig().make_env(
    num_players=2,
    map_type="BASE",
    vps_to_win=10,
    representation="vector",
    invalid_action_reward=-10,
    render_mode="rgb_array",
    auto_play_single_action=True,
)
env = record_video_wrapper(copy.deepcopy(env), "./videos", 1)
params = {
    "num_simulations": 50,
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "action_function": action_as_onehot,
    "n_step": 750,
    "residual_layers": [],
    "dense_layer_widths": [128] * 3,
    "reward_dense_layer_widths": [32],
    "reward_conv_layers": [],
    "chance_dense_layer_widths": [32],
    "chance_conv_layers": [],
    "actor_dense_layer_widths": [32],
    "actor_conv_layers": [],
    "critic_dense_layer_widths": [32],
    "critic_conv_layers": [],
    "to_play_dense_layer_widths": [32],
    "to_play_conv_layers": [],
    "known_bounds": [-1, 1],
    "support_range": None,
    "minibatch_size": 32,
    "replay_buffer_size": 100000,
    "lr_ratio": float("inf"),
    "num_workers": 4,
    "min_replay_buffer_size": 1000,
    "root_dirichlet_alpha": 0.03,
    "learning_rate": 0.001,  # 0.001 or sgd 0.1
    "optimizer": Adam,  # SGD
    "temperature_updates": [150],
    "to_play_loss_factor": 1.0,
    "transfer_interval": 1,
    "gumbel": True,
    "gumbel_m": 16,
    "policy_loss_function": KLDivergenceLoss(),
    "training_steps": 200000,
    "reanalyze_ratio": 0.0,
    "reanalyze_noise": True,  # for gumbel
    "value_loss_factor": 1.0,  # for reanalyze
    "injection_frac": 0.25,
    "reanalyze_method": "mcts",
    "consistency_loss_factor": 0.0,  # might want to be zero for "average state" planning, as initial is known, but one step forward in dynamics has some randomness
    "projector_output_dim": 128,
    "projector_hidden_dim": 128,
    "predictor_output_dim": 128,
    "predictor_hidden_dim": 64,
    "value_prefix": True,
    "lstm_horizon_len": 5,
    "lstm_hidden_size": 64,
    "q_estimation_method": "v_mix",
    "clipnorm": 0,
    "stochastic": True,
    "vqvae_commitment_cost_factor": 0.5,
}
game_config = CatanConfig()
config = MuZeroConfig(config_dict=params, game_config=game_config)


agent = MuZeroAgent(
    env=env,
    config=config,
    name="catan_gumbel_stochastic_value_prefix",
    device="cpu",
    test_agents=[
        CatanPlayerWrapper(RandomPlayer, Color.BLUE),
        CatanPlayerWrapper(AlphaBetaPlayer, Color.BLUE),
    ],
)
agent.checkpoint_interval = 100
agent.test_interval = 2500
agent.test_trials = 10

agent.train()

In [ ]:
agent.run_tests(agent.stats)

In [ ]:
# New SMALLEST SEARCH SPACE, IMPROVED
import sys

import numpy as np

sys.path.append("../../")

from hyperparameter_optimization.hyperopt import save_search_space


import dill as pickle
from hyperopt import hp
from hyperopt.pyll import scope
from utils import CategoricalCrossentropyLoss, MSELoss, generate_layer_widths
import gymnasium as gym
import torch
from muzero.action_functions import action_as_onehot as action_function
from torch.optim import Adam, SGD

# size = 5 * 1 * 1 * 4.0 * 3 * 2.0 * 5 * 1 * 1 = 600

search_space = {
    "kernel_initializer": hp.choice(
        "kernel_initializer",
        [
            "he_uniform",
            "he_normal",
            "glorot_uniform",
            "glorot_normal",
            "orthogonal",
        ],
    ),
    "optimizer": hp.choice(
        "optimizer",
        [
            {
                "optimizer": "adam",
                # "adam_epsilon": 10 ** (-hp.quniform("adam_epsilon", 8, 8 + 1e-8, 2)),
                "adam_epsilon": hp.choice("adam_epsilon", [1e-8]),
                "adam_learning_rate": 10
                ** (-hp.quniform("adam_learning_rate", 3, 3 + 1e-8, 1)),
            },
            # {
            #     "optimizer": "sgd",
            #     "momentum": hp.choice("momentum", [0.0, 0.9]),
            #     "sgd_learning_rate": 10 ** (-hp.quniform("sgd_learning_rate", 1, 3, 1)),
            # },
        ],
    ),
    "conv_layers": hp.choice("conv_layers", [[]]),
    "known_bounds": hp.choice("known_bounds", [[-1, 1]]),
    # "residual_filters": scope.int(
    #     hp.qloguniform("residual_filters", np.log(24), np.log(24) + 1e-8, 8)
    # ),
    # "residual_stacks": scope.int(
    #     hp.qloguniform("residual_stacks", np.log(1), np.log(4), 1)
    # ),
    "residual_layers": hp.choice("residual_layers", [[]]),
    "actor_conv_layers": hp.choice("actor_conv_layers", [[]]),
    "critic_conv_layers": hp.choice("critic_conv_layers", [[]]),
    "reward_conv_layers": hp.choice("reward_conv_layers", [[]]),
    "to_play_conv_layers": hp.choice("to_play_conv_layers", [[]]),
    "output_layer_widths": scope.int(
        hp.quniform("output_layer_widths", 16, 16 + 1e-8, 16)
    ),
    "dense_layer_width": scope.int(2 ** hp.quniform("dense_layer_width", 6, 9, 1)),
    "dense_layers": scope.int(hp.quniform("dense_layers", 1, 1 + 1e-8, 1)),
    "noisy_sigma": hp.choice("noisy_sigma", [0.0]),
    "value_loss_factor": 2 ** hp.quniform("value_loss_factor", 0.0, 0.0 + 1e-8, 1.0),
    "to_play_loss_factor": 2 ** hp.quniform("to_play_loss_factor", 0.0, 1.0, 1.0),
    "root_dirichlet_alpha": 2 ** (hp.quniform("root_dirichlet_alpha", -5, 0, 1.0)),
    "root_exploration_fraction": hp.choice("root_exploration_fraction", [0.25]),
    "num_simulations": scope.int(
        800 * 2 ** hp.quniform("num_simulations", -5, -4 + 1e-8, 1)
    ),
    "temperature_updates": [scope.int(hp.quniform("temperature_updates", 32, 512, 32))],
    "temperatures": hp.choice("temperatures", [[1.0, 0.1]]),
    "temperature_with_training_steps": hp.choice(
        "temperature_with_training_steps", [False]
    ),
    "clip_low_prob": hp.choice("clip_low_prob", [0.0]),
    "pb_c_base": hp.choice("pb_c_base", [19652]),
    "pb_c_init": hp.choice("pb_c_init", [1.25]),
    "value_loss_function": hp.choice("value_loss_function", [MSELoss()]),
    "reward_loss_function": hp.choice("reward_loss_function", [MSELoss()]),
    "policy_loss_function": hp.choice(
        "policy_loss_function", [CategoricalCrossentropyLoss()]
    ),
    "to_play_loss_function": hp.choice(
        "to_play_loss_function", [CategoricalCrossentropyLoss()]
    ),
    "training_steps": scope.int(
        hp.qloguniform("training_steps", np.log(500), np.log(1000), 100)
    ),
    "minibatch_size": scope.int(2 ** (hp.quniform("minibatch_size", 3, 11, 1))),
    "min_replay_buffer_size": scope.int(
        2 ** hp.quniform("min_replay_buffer_size", 5, 10, 1)
    ),
    "replay_buffer_size": scope.int(
        10 ** (hp.quniform("replay_buffer_size", 5, 5 + 1e-8, 1))
    ),
    "unroll_steps": hp.choice("unroll_steps", [5]),
    "n_step": hp.choice("n_step", [750]),
    "clipnorm": hp.choice(
        "clipnorm",
        [0.0, scope.int(10 ** (hp.quniform("clip_val", 0, 2, 1)))],
        # "clipnorm",
        # [0.0],
    ),
    "weight_decay": hp.choice("weight_decay", [1e-4]),
    "per_alpha": hp.quniform("per_alpha", 0.0, 0.0 + 1e-8, 0.5),
    "per_beta": hp.quniform("per_beta", 0.0, 0.0 + 1e-8, 0.5),
    "per_beta_final": hp.quniform("per_beta_final", 0.0, 0.0 + 1e-8, 0.5),
    "per_epsilon": hp.choice("per_epsilon", [1e-4]),
    "action_function": hp.choice("action_function", [action_function]),
    "multi_process": hp.choice(
        "multi_process",
        [
            {
                "multi_process": True,
                "num_workers": scope.int(hp.quniform("num_workers", 2, 4, 1)),
            },
            # {
            #     "multi_process": False,
            #     "games_per_generation": scope.int(
            #         hp.qloguniform("games_per_generation", np.log(8), np.log(32), 8)
            #     ),
            # },
        ],
    ),
    "lr_ratio": hp.choice("lr_ratio", [float("inf"), 0.1]),
    "transfer_interval": scope.int(10 ** hp.quniform("transfer_interval", 0, 1, 1)),
    "vps_to_win": hp.quniform("vps_to_win", 3, 3 + 1e-8, 1),
}

initial_best_config = []

search_space, initial_best_config = save_search_space(search_space, initial_best_config)


def prep_params(params):
    params["dense_layer_widths"] = [params["dense_layer_width"]] * params[
        "dense_layers"
    ]
    del params["dense_layer_width"]
    del params["dense_layers"]
    if params["output_layer_widths"] != 0:
        params["actor_dense_layer_widths"] = [params["output_layer_widths"]]
        params["critic_dense_layer_widths"] = [params["output_layer_widths"]]
        params["reward_dense_layer_widths"] = [params["output_layer_widths"]]
        params["to_play_dense_layer_widths"] = [params["output_layer_widths"]]
    else:
        params["actor_dense_layer_widths"] = []
        params["critic_dense_layer_widths"] = []
        params["reward_dense_layer_widths"] = []
        params["to_play_dense_layer_widths"] = []
    del params["output_layer_widths"]

    if params["multi_process"]["multi_process"] == True:
        params["num_workers"] = params["multi_process"]["num_workers"]
        params["multi_process"] = True
    else:
        params["games_per_generation"] = params["multi_process"]["games_per_generation"]
        params["multi_process"] = False

    if params["optimizer"]["optimizer"] == "adam":
        params["adam_epsilon"] = params["optimizer"]["adam_epsilon"]
        params["learning_rate"] = params["optimizer"]["adam_learning_rate"]
        params["optimizer"] = Adam
    elif params["optimizer"]["optimizer"] == "sgd":
        params["momentum"] = params["optimizer"]["momentum"]
        params["learning_rate"] = params["optimizer"]["sgd_learning_rate"]
        params["optimizer"] = SGD

    if isinstance(params["clipnorm"], dict):
        params["clipnorm"] = params["clipnorm"]["clipval"]
    params["support_range"] = None

    return params

In [ ]:
from game_configs.catan_config import CatanConfig
import torch
from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)


def play_game(player1, player2):

    env = CatanConfig().make_env()
    with torch.no_grad():  # No gradient computation during testing
        # Reset environment
        env.reset()
        state, reward, termination, truncation, info = env.last()
        done = termination or truncation
        agent_id = env.agent_selection
        current_player = env.agents.index(agent_id)
        # state, info = process_petting_zoo_obs(state, info, current_player)
        agent_names = env.agents.copy()

        episode_length = 0
        while not done and episode_length < 1000:  # Safety limit
            # Get current agent and player
            if current_player == 0:
                prediction = player1.predict(state, info, env=env, temperature=0.1)
                action = player1.select_actions(prediction, info).item()
            else:
                prediction = player2.predict(state, info, env=env, temperature=0.1)
                action = player2.select_actions(prediction, info).item()

            # Step environment
            env.step(action)
            state, reward, termination, truncation, info = env.last()
            agent_id = env.agent_selection
            current_player = env.agents.index(agent_id)
            # state, info = process_petting_zoo_obs(state, info, current_player)
            done = termination or truncation
            episode_length += 1
        print(env.rewards)
        return env.rewards["player_0"]

In [ ]:
from agents.catan_player_wrapper import CatanPlayerWrapper
from hyperparameter_optimization.hyperopt import (
    marl_objective,
    set_marl_config,
    MarlHyperoptConfig,
)
from hyperopt import atpe, tpe, fmin, space_eval
from hyperopt.exceptions import AllTrialsFailed

from agents.muzero_agent_torch import MuZeroAgent
from agent_configs import MuZeroConfig
from utils.utils import record_video_wrapper

search_space_path, initial_best_config_path = (
    "search_space.pkl",
    "best_config.pkl",
)
# search_space = pickle.load(open(search_space_path, "rb"))
# initial_best_config = pickle.load(open(initial_best_config_path, "rb"))
file_name = "catan_muzero"
max_trials = 64
trials_step = 24  # how many additional trials to do after loading the last ones

from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer

import pandas as pd
import random
from tqdm import tqdm
import sys
import dill as pickle

sys.path.append("../../")
from elo.elo import StandingsTable

games_per_pair = 10
try:
    players = pickle.load(open("./tictactoe_players.pkl", "rb"))
    table = pickle.load(open("./tictactoe_table.pkl", "rb"))
    print(table.bayes_elo())
    print(table.get_win_table())
    print(table.get_draw_table())
except:
    players = []
    table = StandingsTable([], start_elo=1000)


set_marl_config(
    MarlHyperoptConfig(
        file_name=file_name,
        eval_method="test_agents_elo",
        best_agent=CatanPlayerWrapper(AlphaBetaPlayer, Color.BLUE),
        make_env=CatanConfig().make_env,
        prep_params=prep_params,
        agent_class=MuZeroAgent,
        agent_config=MuZeroConfig,
        game_config=CatanConfig,
        games_per_pair=10,
        num_opps=1,  # not used
        table=table,  # not used
        play_game=play_game,
        checkpoint_interval=1,
        test_interval=1000,
        test_trials=100,
        test_agents=[
            CatanPlayerWrapper(RandomPlayer, Color.BLUE),
            CatanPlayerWrapper(AlphaBetaPlayer, Color.BLUE),
        ],
        test_agent_weights=[1.0, 2.0],
        device="cpu",
    )
)

try:  # try to load an already saved trials object, and increase the max
    trials = pickle.load(open(f"./{file_name}_trials.p", "rb"))
    print("Found saved Trials! Loading...")
    max_trials = len(trials.trials) + trials_step
    print(
        "Rerunning from {} trials to {} (+{}) trials".format(
            len(trials.trials), max_trials, trials_step
        )
    )
except:  # create a new trials object and start searching
    print("No saved Trials! Starting from scratch.")
    trials = None

best = fmin(
    fn=marl_objective,  # Objective Function to optimize
    space=search_space,  # Hyperparameter's Search Space
    algo=atpe.suggest,  # Optimization algorithm (representative TPE)
    max_evals=max_trials,  # Number of optimization attempts
    trials=trials,  # Record the results
    # early_stop_fn=no_progress_loss(5, 1),
    trials_save_file=f"./{file_name}_trials.p",
    points_to_evaluate=initial_best_config,
    show_progressbar=False,
)
print(best)
best_trial = space_eval(search_space, best)
# gc.collect()

RAINBOW

In [ ]:
import sys

# sys.path.append("/content/rl-research")
sys.path.append("../..")
import gymnasium as gym
import torch

from wrappers import CatanatronWrapper
from utils import CategoricalCrossentropyLoss, KLDivergenceLoss


from agents.rainbow_agent import RainbowAgent
from agent_configs import RainbowConfig
from game_configs.catan_config import SinglePlayerCatanConfig
from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer

config_dict = {
    "dense_layer_widths": [512],
    "value_hidden_layers_widths": [512],  #
    "advatage_hidden_layers_widths": [512],  #
    "adam_epsilon": 1e-8,
    "learning_rate": 0.001,
    "training_steps": 30000,
    "per_epsilon": 1e-6,
    "per_alpha": 0.5,
    "per_beta": 0.5,
    "minibatch_size": 64,
    "replay_buffer_size": 50000,
    "min_replay_buffer_size": 5000,
    "transfer_interval": 100,
    "n_step": 9,
    "kernel_initializer": "orthogonal",
    "loss_function": KLDivergenceLoss(),
    "clipnorm": 0.0,
    "discount_factor": 0.99,  # or 0.999 or even 0.9999 not 0.99 < this makes the start of the game possibly 0.05 after bootstrapping
    "atom_size": 51,
    "replay_interval": 16,
}
game_config = SinglePlayerCatanConfig()
game_config.min_score = -1000
game_config.max_score = 1010
config = RainbowConfig(config_dict, game_config)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
import catanatron.gym
import gymnasium as gym

from catanatron.gym.envs.catanatron_env import simple_reward
from catanatron.gym.utils import get_tournament_total_return
from catanatron.gym.utils import get_actual_victory_points


def tournament_reward_function(game, p0_color):
    winning_color = game.winning_color()
    if p0_color == winning_color:
        return get_tournament_total_return(game, p0_color)
    elif winning_color is None:
        return 0
    else:
        return get_tournament_total_return(game, p0_color)


env = CatanatronWrapper(
    gym.make(
        "catanatron/Catanatron-v0",
        config={
            "enemies": [RandomPlayer(Color.RED)],
            "invalid_action_reward": -10,
            "map_type": "BASE",
            "vps_to_win": 3,
            "representation": "vector",
            "reward_function": tournament_reward_function,
        },
    )
)
agent = RainbowAgent(env, config, "rainbow-catan-3vps", device)
agent.checkpoint_interval = 100
agent.test_interval = 1000
agent.test_trials = 50
agent.train()

In [ ]:
# shared network but not shared buffer?
# 1 vs 2 minibatches
import sys

sys.path.append("../../")
from agents.nfsp_agent_clean import NFSPDQN
from agent_configs import NFSPDQNConfig
from game_configs import CatanConfig
from utils import KLDivergenceLoss, CategoricalCrossentropyLoss, HuberLoss, MSELoss
from torch.optim import Adam, SGD

from wrappers import ActionMaskInInfoWrapper, FrameStackWrapper

config_dict = {
    "shared_networks_and_buffers": False,
    "training_steps": 500000,
    "anticipatory_param": 0.1,
    "replay_interval": 2048,  #
    "num_minibatches": 1,  # or 2, could be 2 minibatches per network, or 2 minibatches (1 for each network/player)
    "learning_rate": 0.1,
    "momentum": 0.0,
    "optimizer": SGD,
    "loss_function": MSELoss(),
    "min_replay_buffer_size": 5000,
    "minibatch_size": 1024,
    "replay_buffer_size": 1e5,
    "transfer_interval": 100,
    "residual_layers": [],
    "conv_layers": [],
    "dense_layer_widths": [1024],
    "value_hidden_layer_widths": [],
    "advantage_hidden_layer_widths": [],
    "noisy_sigma": 0.0,
    "eg_epsilon": 0.12,
    # "eg_epsilon_final": 0.06,
    "eg_epsilon_decay_type": "inverse_sqrt",
    "eg_epsilon_decay_final_step": 0,
    "sl_learning_rate": 0.005,
    "sl_momentum": 0.0,
    # "sl_weight_decay": 1e-9,
    # "sl_clipnorm": 1.0,
    "sl_optimizer": SGD,
    "sl_loss_function": CategoricalCrossentropyLoss(),
    "sl_min_replay_buffer_size": 5000,
    "sl_minibatch_size": 1024,
    "sl_replay_buffer_size": 100000,
    "sl_residual_layers": [],
    "sl_conv_layers": [],
    "sl_dense_layer_widths": [1024, 1024, 1024],
    "sl_clip_low_prob": 0.0,
    "per_alpha": 0.5,
    "per_beta": 1.0,
    "per_beta_final": 1.0,
    "per_epsilon": 0.00001,
    "n_step": 9,
    "atom_size": 1,
    "dueling": True,
    "clipnorm": 10.0,
    "sl_clipnorm": 10.0,
}
config = NFSPDQNConfig(
    config_dict=config_dict,
    game_config=CatanConfig(),
)
config.save_intermediate_weights = False
from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)

env = catan_env(
    num_players=2,
    map_type="BASE",
    vps_to_win=10,
    representation="vector",
    invalid_action_reward=-10,
)

env = ActionMaskInInfoWrapper(env)
env = FrameStackWrapper(env, 4, channel_first=False)

agent = NFSPDQN(env, config, name="nfsp-catan", device="cpu")
agent.checkpoint_interval = 100
agent.test_interval = 1000
agent.test_trials = 100
agent.train()

In [ ]:
import sys
import catanatron.gym

sys.path.append("../../")
import gymnasium as gym
import random

env = gym.make("catanatron/Catanatron-v0")
observation, info = env.reset()
for _ in range(1000):
    # your agent here (this takes random actions)
    action = random.choice(info["valid_actions"])

    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    if done:
        observation, info = env.reset()
env.close()

In [ ]:
from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer

# Instantiate two random players
player1 = RandomPlayer(Color.RED)
player2 = RandomPlayer(Color.BLUE)

# Create a 2-player game (you can fill remaining slots with random agents if needed)
players = [player1, player2]
game = Game(players)

winner = game.play()
print(f"Winner: {winner}")


def play_game(player1, player2):
    player1 = player1(Color.RED)
    player2 = player2(Color.BLUE)
    game = Game([player1, player2])
    winner = game.play()

    if winner == Color.RED:
        return 1
    elif winner == Color.BLUE:
        return -1
    else:
        return 0

In [ ]:
import pandas as pd
import random
from tqdm import tqdm
import sys

sys.path.append("../../")
from elo.elo import StandingsTable

players = [
    RandomPlayer,
    MCTSPlayer,
    AlphaBetaPlayer,
    # GreedyPlayoutsPlayer,
    VictoryPointPlayer,
    WeightedRandomPlayer,
    # ValueFunctionPlayer,
]
games_per_pair = 10

player_names = [p.__name__ for p in players]
table = StandingsTable(player_names, start_elo=1000)


def play_1v1_tournament(players, games_per_pair, play_game):
    tournament_results = []
    for player1 in players:
        results = play_matches(player1, players, games_per_pair, play_game)
        tournament_results.extend(results)
    tournament_results = pd.DataFrame(
        tournament_results, columns=["player1", "player2", "result"]
    )
    return tournament_results


def play_matches(player1, players, games_per_pair, play_game):
    results = []
    for opponent in players:
        if opponent != player1:
            for _ in range(games_per_pair // 2):
                print(f"Playing {player1.__name__} vs {opponent.__name__} game {_+1}")
                result = play_game(player1, opponent)
                results.append((player1.__name__, opponent.__name__, result))

    for opponent in players:
        if opponent != player1:
            for _ in range(games_per_pair // 2):
                print(f"Playing {opponent.__name__} vs {player1.__name__} game {_+1}")
                result = play_game(opponent, player1)
                results.append(
                    (
                        player_names[players.index(opponent)],
                        player_names[players.index(player1)],
                        result,
                    )
                )
    table.add_results_from_array(results)
    print(table.bayes_elo())
    return results

In [ ]:
import pickle

print(table.bayes_elo())
print(table.get_win_table())
print(table.get_draw_table())
file = "catan_1v1_tournament_results.pkl"
pickle.dump(table, open(file, "wb"))

In [ ]:
from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer

table.add_player(GreedyPlayoutsPlayer.__name__)
players.append(GreedyPlayoutsPlayer) if GreedyPlayoutsPlayer not in players else None

play_matches(GreedyPlayoutsPlayer, players, games_per_pair * 2, play_game)

In [ ]:
results = play_1v1_tournament(players, games_per_pair, play_game)


# table.add_results_from_dataframe(results)  # Adding multiple results
print(table.bayes_elo())

In [ ]:
# Test a petting zoo environment to see if it has all the functions and attributes needed
import sys

sys.path.append("../../")
from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)
from agents.catan_player_wrapper import CatanPlayerWrapper
from catanatron.players.minimax import AlphaBetaPlayer
from utils.utils import record_video_wrapper
from catanatron import Game, RandomPlayer, Color
from game_configs.catan_config import CatanConfig
from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)

# env = catan_env(
#     render_mode="rgb_array",
#     num_players=2,
#     map_type="BASE",
#     vps_to_win=10,
#     representation="vector",
#     invalid_action_reward=-1,
#     auto_play_single_action=False,
# )


env = CatanConfig().make_env(
    num_players=2,
    map_type="BASE",
    vps_to_win=10,
    representation="vector",
    invalid_action_reward=-10,
    render_mode="rgb_array",
    auto_play_single_action=False,
)
env = record_video_wrapper(env, "./videos", 1)
env.reset()

ab_player = CatanPlayerWrapper(AlphaBetaPlayer, Color.RED)

for i in range(100):
    state, reward, termination, truncation, info = env.last()
    prediction = ab_player.predict(state, info, env)
    action = ab_player.select_actions(prediction, info).item()
    print(action)
    print(info)
    env.step(action)
env.close()

In [ ]:
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)

env = catan_env(
    render_mode="rgb_array",
    num_players=2,
    map_type="BASE",
    vps_to_win=10,
    representation="vector",
    invalid_action_reward=-1,
)

# Set metadata fps for 'human' if desired (not required for rgb_array)
env.metadata["render_fps"] = 10

# Reset the environment (seed optional)
env.reset(seed=0)
print(env.last())
print("Starting agent selection:", env.agent_selection)

# # Render one frame as an RGB array, then plot it
# frame = env.render()  # will return a HxWx3 numpy array if renderer implemented
# if frame is None:
#     print(
#         "Renderer returned None. Make sure env.render(mode='rgb_array') is implemented."
#     )
# else:
#     plt.figure(figsize=(8, 6))
#     plt.imshow(frame)
#     plt.axis("off")
#     plt.show()

# Setup matplotlib figure
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.axis("off")

print("Starting Catan Game Visualization...")
print("=" * 50)

# Run game for a few steps
step_count = 0
max_steps = 20

while step_count < max_steps:
    # Render current state
    rgb_array = env.render()
    print(env.last())

    if rgb_array is not None:
        # Display in notebook
        # clear_output(wait=True)
        ax.clear()
        ax.imshow(rgb_array)
        ax.axis("off")
        ax.set_title(f"Catan Game - Step {step_count}", fontsize=16, fontweight="bold")
        plt.tight_layout()
        display(fig)

        # Print game info
        current_agent = env.agent_selection
        print(f"\nStep {step_count}")
        print(f"Current Player: {current_agent}")

        if hasattr(env, "game") and hasattr(env.game, "state"):
            print(f"Turn Number: {env.game.state.num_turns}")
        print("-" * 50)

    # Check if game is done
    if env.terminations[env.agent_selection] or env.truncations[env.agent_selection]:
        print("\nGame Over!")
        break

    # Get valid actions
    obs = env.observe(env.agent_selection)
    action_mask = obs["action_mask"]
    valid_actions = np.where(action_mask == 1)[0]

    if len(valid_actions) > 0:
        # Take a random valid action
        action = np.random.choice(valid_actions)
        env.step(action)
    else:
        print("No valid actions available!")
        break

    step_count += 1
    time.sleep(0.5)  # Pause to see each frame

print("\nTest completed!")
plt.close()

In [ ]:
from IPython.display import display, clear_output
from custom_gym_envs.envs.catan import ActionType
import sys

sys.path.append("../../")
from agents.catan_player_wrapper import ACTIONS_ARRAY

n_steps = 25
delay = 0.25  # seconds between frames

for step in range(n_steps):
    # If the current agent is terminated/truncated, we still call env_instance.step per PettingZoo convention:
    current_agent = env.agent_selection

    # get observation for current agent
    obs = env.observe(current_agent)
    action_mask = obs["action_mask"]

    legal_indices = np.nonzero(action_mask)[0]
    if len(legal_indices) == 0:
        # no legal moves available (should rarely happen). Choose end-turn fallback if available:
        try:
            end_turn_index = ACTIONS_ARRAY.index((ActionType.END_TURN, None))
            action_choice = end_turn_index
        except Exception:
            # fallback to 0
            action_choice = 0
    else:
        action_choice = int(random.choice(legal_indices))

    # Step the env with the chosen action
    env.step(action_choice)

    # Render and display frame
    frame = env.render()
    if frame is not None:
        clear_output(wait=True)
        plt.figure(figsize=(8, 6))
        plt.imshow(frame)
        plt.title(f"Step {step+1} - Agent: {current_agent} - Action: {action_choice}")
        plt.axis("off")
        display(plt.gcf())
        plt.close()
    else:
        print("No frame returned (render returned None)")
    time.sleep(delay)

# After rollout, print summary
print("Done. Final agent selection:", env.agent_selection)